In [2]:
import sys
from pathlib import Path
base_dir = Path.cwd().parent.resolve()
sys.path.append(str(base_dir))
from decorators import Dispatcher
from validator import TypeChecker, ValidationError
from datetime import date

In [3]:
class Person:
    first_name = TypeChecker(str, min_length=1, max_length=8, required=True)
    last_name = TypeChecker(str, min_length=1, max_length=20, required=True)
    dob = TypeChecker(date, min_length=1, max_length=20, required=True)
    eye_color = TypeChecker("enum", enum_list=['blue', 'green', 'hazel'], required=False)

    def __init__(self, first_name, last_name, dob, eye_color=None):
        self.first_name = first_name
        self.last_name = last_name
        self.dob = dob
        self.eye_color = eye_color

In [4]:
p1 = Person('Ioannis', 'Mougios','19950727', 'brown') # incorrect eye-color
p1 = Person('Ioannis', 'Mougios',dob=None, eye_color= 'hazel') # missing date
p2 = Person('Joe', None, '2000-05-30') # Missing last_name
p3 = Person('Joe', None, '2000/05/30', 'blue') # No logging for missing last name , wrong date format,
p4 = Person('Margarita', 'Rose', date(2000,1,1)) # exceeds max length in first name
p5 = Person('Eric','Boo', '20000101','green') # Correct

In [5]:
ValidationError.get_errors()

mappingproxy({'eye_color': [validator.ValidationError("Acceptable values for attribute eye_color are ['blue', 'green', 'hazel']. Invalid value: brown")],
              'dob': [validator.ValidationError('Field dob is mandatory. Ensure there are no missing values'),
               validator.ValidationError('dob must match one of the formats: %Y%m%d, %Y-%m-%d. Invalid value: 2000/05/30')],
              'last_name': [validator.ValidationError('Field last_name is mandatory. Ensure there are no missing values')],
              'first_name': [validator.ValidationError('Field first_name cannot have more than 8 characters. Invalid value: Margarita')]})

In [6]:
from pprint import pprint
print(ValidationError.to_json())

{
    "eye_color": [
        "Acceptable values for attribute eye_color are ['blue', 'green', 'hazel']. Invalid value: brown"
    ],
    "dob": [
        "Field dob is mandatory. Ensure there are no missing values",
        "dob must match one of the formats: %Y%m%d, %Y-%m-%d. Invalid value: 2000/05/30"
    ],
    "last_name": [
        "Field last_name is mandatory. Ensure there are no missing values"
    ],
    "first_name": [
        "Field first_name cannot have more than 8 characters. Invalid value: Margarita"
    ]
}


In [18]:
ValidationError.clear_errors()

In [19]:
ValidationError.get_errors()

mappingproxy({})

### Validation function can be overwritten ...

In [20]:
class UnregisteredCase(KeyError):
    """Default error when the validate function cannot perform the dispatching based on an argument"""

import re


@Dispatcher
def validate(*args, **kwargs):
    return UnregisteredCase('No register function for the case of {}'.format(args[0]))

@validate.register('email')
def is_valid_email(type_, email: str, **kwargs) -> tuple:

    email_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'

    if not bool(re.match(email_pattern, email)):
        return False, ValidationError(error_message=f'Invalid email address {email}',
                                      field_name=kwargs['attr_name'])

    return True, ''

@validate.register(str)
def validate_string(type_, value, attr_name, min_length, max_length, **kwargs):
    if value is None:
        return True, ""
    if not isinstance(value, str):
        return False, ValidationError(f'Field {attr_name} is type of str. Invalid value: {value}',
                                      field_name=attr_name)
    if min_length and len(value) <= min_length:
        return False, ValidationError(f"Field {attr_name} must have at least {min_length} characters. Invalid value: {value}",
                                      field_name=attr_name)
    if max_length and len(value) >= max_length:
        return False, ValidationError(f"Field {attr_name} cannot have more than {max_length} characters. Invalid value: {value}",
                                      field_name=attr_name)
    return True, ""

TypeChecker.set_validator_function(validate)

<staticmethod(<decorators.dispatcher.Dispatcher object at 0x128162e10>)>

In [21]:
class Person:
    name = TypeChecker(str, min_length=1, max_length=20, required=True)
    email = TypeChecker('email', required=True)

    def __init__(self, name, email):
        self.name = name
        self.email = email

In [22]:
p1 = Person('Ioannis',"test@example.com")
p2 = Person('Ioannis'*6,"invalid-email@")

In [23]:
ValidationError.get_errors()


mappingproxy({'name': [validator.ValidationError('Field name cannot have more than 20 characters. Invalid value: IoannisIoannisIoannisIoannisIoannisIoannis')],
              'email': [validator.ValidationError('Invalid email address invalid-email@')]})

In [13]:
# emails = ["test@example.com", "invalid-email@", "user@domain", "hello@sub.domain.com"]
# for email in emails:
#     print(f"{email}: {validate('email', email)}")